# Deep Reinforcement Learning for Robotic Systems 

## Synopsis

This notebook outlines the modelling and integration of the **[Proximal Policy Optimisation](http://arxiv.org/abs/1707.06347)** algorithm on an **inverted double pendulum** as a baseline study into advanced astrodynamical control systems, such as docking and berthing of spacecraft, and rocket initialisation stabilisation. 

--------

Produced by *[Mughees Asif](https://github.com/mughees-asif)*, under the supervision of [Dr. Angadh Nanjangud](https://www.sems.qmul.ac.uk/staff/a.nanjangud) (Lecturer in Aerospace/Spacecraft Engineering @ [Queen Mary, University of London](https://www.sems.qmul.ac.uk/)).



## 1. Overview

Proximal Policy Optimisation is a deep reinforcement learning algorithm developed by [OpenAI](https://spinningup.openai.com/en/latest/algorithms/ppo.html). It has proven to be successful in a variety of tasks ranging from enabling robotic systems in complex environments, to developing proficiency in computer gaming by using stochastic mathematical modelling to simulate real-life decision making. For the purposes of this research, the algorithm will be implemented to vertically stablise an inverted double pendulum, which is widely used in industry as a benchmark to validate the veracity of next-generation intelligent algorithms.

## 2. Model description

An inverted double pendulum is a characteristic example of a simple-to-build, non-linear, and chaotic mechanical system that has been widely studied in the fields of Robotics, Aerospace, Biomedical, Mechanical Engineering, and Mathematical Analysis.

<img src="images/dip_fbd.png" width="350" />

## 3. Variables

<img src="images/variables.png" width="400" />

## 4. Governing equations of motion

The following section utilises the [SymPy](https://www.sympy.org/en/index.html) package to derive the governing equations of motion. 

### 4.1. Basic modelling

<img src="images/dip_fbd_radius.png" width="300" />

In [1]:
import sympy

# initiliase variables
t = sympy.symbols('t')        # time
m = sympy.symbols('m')        # mass of the cart
l = sympy.symbols('l')        # length of the pendulums, l_1 = l_2 = l
M = sympy.symbols('M')        # mass of the pendulums, M_1 = M_2 = M
I = sympy.symbols('I')        # moment of inertia
g = sympy.symbols('g')        # gravitational constant, 9.81 m/s^2
F = sympy.symbols('F')        # force applied to the cart

x = sympy.Function('x')(t)    # |
Θ = sympy.Function('Θ')(t)    # | --- functions of (t)
Φ = sympy.Function('Φ')(t)    # |

# cart
x_dot = x.diff(t)             # velocity

# pendulum(s) 
x_1 = x + (l*sympy.sin(Θ))    # | --- position
x_2 = l*sympy.cos(Θ)          # | 

v_1 = x_1 + l*sympy.sin(Φ)                                             # |
v_2 = x_2 + l*sympy.cos(Φ)                                             # | --- linear velocity
v_3 = sympy.sqrt(sympy.simplify(x_1.diff(t)**2 + x_2.diff(t)**2))      # |  
v_4 = sympy.sqrt(sympy.simplify(v_1.diff(t)**2 + v_2.diff(t)**2))      # |

Θ_dot = Θ.diff(t)             # | --- angular velocity
Φ_dot = Φ.diff(t)             # |

### 4.2. Kinetic and Potential Energy

In [2]:
# kinetic energy 
K = 0.5*((m*x_dot**2) + M*(v_3**2 + v_4**2) + I*(Θ_dot**2 + Φ_dot**2))

# potential energy 
P = M*g*l*(2*sympy.cos(Θ) + sympy.cos(Φ))

In [3]:
print('------------------------------\nThe kinetic energy, K, of the system:\n------------------------------')
K

------------------------------
The kinetic energy, K, of the system:
------------------------------


0.5*I*(Derivative(Θ(t), t)**2 + Derivative(Φ(t), t)**2) + 0.5*M*(2*l**2*cos(Θ(t) - Φ(t))*Derivative(Θ(t), t)*Derivative(Φ(t), t) + 2*l**2*Derivative(Θ(t), t)**2 + l**2*Derivative(Φ(t), t)**2 + 4*l*cos(Θ(t))*Derivative(x(t), t)*Derivative(Θ(t), t) + 2*l*cos(Φ(t))*Derivative(x(t), t)*Derivative(Φ(t), t) + 2*Derivative(x(t), t)**2) + 0.5*m*Derivative(x(t), t)**2

In [4]:
print('------------------------------\nThe potential energy, P, of the system:\n------------------------------')
P

------------------------------
The potential energy, P, of the system:
------------------------------


M*g*l*(2*cos(Θ(t)) + cos(Φ(t)))

### 4.3. The Lagrangian

The action $S$ of the cart (movement; left, right) is mathematically defined as:

$$S = \int_{t_{0}}^{t_{1}} K - P \,dt$$

but, $L = K - P$

$$\therefore S = \int_{t_{0}}^{t_{1}} L \,dt$$

In [5]:
# the lagrangian
L = K - P

print('------------------------------\nThe Lagrangian of the system is:\n------------------------------')
L

------------------------------
The Lagrangian of the system is:
------------------------------


0.5*I*(Derivative(Θ(t), t)**2 + Derivative(Φ(t), t)**2) - M*g*l*(2*cos(Θ(t)) + cos(Φ(t))) + 0.5*M*(2*l**2*cos(Θ(t) - Φ(t))*Derivative(Θ(t), t)*Derivative(Φ(t), t) + 2*l**2*Derivative(Θ(t), t)**2 + l**2*Derivative(Φ(t), t)**2 + 4*l*cos(Θ(t))*Derivative(x(t), t)*Derivative(Θ(t), t) + 2*l*cos(Φ(t))*Derivative(x(t), t)*Derivative(Φ(t), t) + 2*Derivative(x(t), t)**2) + 0.5*m*Derivative(x(t), t)**2

### 4.4. The Euler-Lagrange equations

The standard [Euler-Lagrange equation](https://www.ucl.ac.uk/~ucahmto/latex_html/chapter2_latex2html/node5.html) is:

$$\frac{d}{dt}\frac{\partial L}{\partial \dot{x}} - \frac{\partial L}{\partial x} = 0$$

To introduce the generalised force acting on the cart, the [Lagrange-D'Alembert Principle](https://en.wikipedia.org/wiki/D%27Alembert%27s_principle) is used:

$$\frac{d}{dt}\frac{\partial L}{\partial \dot{x}} - \frac{\partial L}{\partial x} = Q^{P}$$

Therefore, for a three-dimensional _working_ system, the equations of motion can be derived as:

$$\frac{d}{dt}\frac{\partial L}{\partial \dot{x}} - \frac{\partial L}{\partial x} = F - \dot x$$
$$\frac{d}{dt}\frac{\partial L}{\partial \dot{\theta}} - \frac{\partial L}{\partial \theta} = 0$$
$$\frac{d}{dt}\frac{\partial L}{\partial \dot{\phi}} - \frac{\partial L}{\partial \phi} = 0$$

In [6]:
# euler-lagrange formulation
euler_1 = sympy.Eq((L.diff(x_dot).diff(t) - L.diff(x)).simplify().expand().collect(x.diff(t, t)), F - x.diff(t))
euler_2 = sympy.Eq((L.diff(Θ_dot).diff(t) - L.diff(Θ)).simplify().expand().collect(Θ.diff(t, t)), 0)
euler_3 = sympy.Eq((L.diff(Φ_dot).diff(t) - L.diff(Φ)).simplify().expand().collect(Φ.diff(t, t)), 0)

print('------------------------------\nThe Euler-Lagrange equations:\n------------------------------\n1.')
euler_1

------------------------------
The Euler-Lagrange equations:
------------------------------
1.


Eq(-2*M*l*sin(Θ(t))*Derivative(Θ(t), t)**2 - M*l*sin(Φ(t))*Derivative(Φ(t), t)**2 + 2*M*l*cos(Θ(t))*Derivative(Θ(t), (t, 2)) + M*l*cos(Φ(t))*Derivative(Φ(t), (t, 2)) + (2*M + 1.0*m)*Derivative(x(t), (t, 2)), F - Derivative(x(t), t))

In [7]:
print('2.')
euler_2

2.


Eq(-2.0*M*g*l*sin(Θ(t)) + 1.0*M*l**2*sin(Θ(t) - Φ(t))*Derivative(Φ(t), t)**2 + 1.0*M*l**2*cos(Θ(t) - Φ(t))*Derivative(Φ(t), (t, 2)) + 2.0*M*l*cos(Θ(t))*Derivative(x(t), (t, 2)) + (1.0*I + 2.0*M*l**2)*Derivative(Θ(t), (t, 2)), 0)

In [8]:
print('3.')
euler_3

3.


Eq(-1.0*M*g*l*sin(Φ(t)) - 1.0*M*l**2*sin(Θ(t) - Φ(t))*Derivative(Θ(t), t)**2 + 1.0*M*l**2*cos(Θ(t) - Φ(t))*Derivative(Θ(t), (t, 2)) + 1.0*M*l*cos(Φ(t))*Derivative(x(t), (t, 2)) + (1.0*I + 1.0*M*l**2)*Derivative(Φ(t), (t, 2)), 0)

### 4.5. Linearisation and acceleration

[Hartman-Grobman theorem](https://en.wikipedia.org/wiki/Hartman%E2%80%93Grobman_theorem)

The pendulum will achieve equilibrium when vertical, i.e. $\theta=0$ & $\phi=0$:

$$\sin(\theta)=\theta, \quad \cos(\theta)=1, \quad \dot\theta^{2}=0$$

$$\sin(\phi)=\phi, \quad \cos(\phi)=1, \quad \dot\phi^{2}=0$$

$$\sin(\theta - \phi)=\theta - \phi, \quad\quad \cos(\theta - \phi)=1$$

In [9]:
# linearise the system
matrix = [(sympy.sin(Θ), Θ), (sympy.cos(Θ), 1), (Θ_dot**2, 0), 
         (sympy.sin(Φ), Φ), (sympy.cos(Φ), 1), (Φ_dot**2, 0),
         (sympy.sin(Θ - Φ), Θ - Φ), (sympy.cos(Θ - Φ), 1)]

linear_1 = euler_1.subs(matrix)
linear_2 = euler_2.subs(matrix)
linear_3 = euler_3.subs(matrix)

print('------------------------------\nThe linear equations are: \n------------------------------\n1.')
linear_1

------------------------------
The linear equations are: 
------------------------------
1.


Eq(2*M*l*Derivative(Θ(t), (t, 2)) + M*l*Derivative(Φ(t), (t, 2)) + (2*M + 1.0*m)*Derivative(x(t), (t, 2)), F - Derivative(x(t), t))

In [10]:
print('2. ')
linear_2

2. 


Eq(-2.0*M*g*l*Θ(t) + 1.0*M*l**2*Derivative(Φ(t), (t, 2)) + 2.0*M*l*Derivative(x(t), (t, 2)) + (1.0*I + 2.0*M*l**2)*Derivative(Θ(t), (t, 2)), 0)

In [11]:
print('3. ')
linear_3

3. 


Eq(-1.0*M*g*l*Φ(t) + 1.0*M*l**2*Derivative(Θ(t), (t, 2)) + 1.0*M*l*Derivative(x(t), (t, 2)) + (1.0*I + 1.0*M*l**2)*Derivative(Φ(t), (t, 2)), 0)

In [12]:
# simplify for linear and angular acceleration
final_equations = sympy.linsolve([linear_1, linear_2, linear_3], [x.diff(t, t), Θ.diff(t, t), Φ.diff(t, t)])

x_ddot = final_equations.args[0][0].expand().collect((Θ, Θ_dot, x, x_dot, Φ, Φ_dot, F)).simplify()
Θ_ddot = final_equations.args[0][1].expand().collect((Θ, Θ_dot, x, x_dot, Φ, Φ_dot, F)).simplify()
Φ_ddot = final_equations.args[0][2].expand().collect((Θ, Θ_dot, x, x_dot, Φ, Φ_dot, F)).simplify()

print('------------------------------\nx_acceleration:\n------------------------------')
x_ddot      

------------------------------
x_acceleration:
------------------------------


(F*(4.0*I**2 + 12.0*I*M*l**2 + 4.0*M**2*l**4) - 4.0*I*M**2*g*l**2*Φ(t) - M**2*g*l**2*(16.0*I + 8.0*M*l**2)*Θ(t) - (4.0*I**2 + 12.0*I*M*l**2 + 4.0*M**2*l**4)*Derivative(x(t), t))/(8.0*I**2*M + 4.0*I**2*m + 4.0*I*M**2*l**2 + 12.0*I*M*l**2*m + 4.0*M**2*l**4*m)

In [13]:
print('------------------------------\nΘ_acceleration:\n------------------------------')
Θ_ddot

------------------------------
Θ_acceleration:
------------------------------


M*l*(-F*(8.0*I + 4.0*M*l**2) - 4.0*M*g*l**2*m*Φ(t) + g*(16.0*I*M + 8.0*I*m + 8.0*M**2*l**2 + 8.0*M*l**2*m)*Θ(t) + (8.0*I + 4.0*M*l**2)*Derivative(x(t), t))/(8.0*I**2*M + 4.0*I**2*m + 4.0*I*M**2*l**2 + 12.0*I*M*l**2*m + 4.0*M**2*l**4*m)

In [14]:
print('------------------------------\nΦ_acceleration:\n------------------------------')
Φ_ddot         

------------------------------
Φ_acceleration:
------------------------------


M*l*(-1.0*F*I + 1.0*I*Derivative(x(t), t) - 2.0*M*g*l**2*m*Θ(t) + g*(2.0*I*M + 1.0*I*m + 2.0*M*l**2*m)*Φ(t))/(2.0*I**2*M + 1.0*I**2*m + 1.0*I*M**2*l**2 + 3.0*I*M*l**2*m + 1.0*M**2*l**4*m)

## 5. Proximal Policy Optimisation

### 5.1. Overview[<sup>1</sup>](#fn1)
 
 * State-of-the-art Policy Gradient method.
 * An on-policy algorithm.
 * Can be used for environments with either discrete or continuous action spaces.
 * **PPO-Clip** doesn’t have a KL-divergence term in the objective and doesn’t have a constraint at all. Instead relies on specialized clipping in the objective function to remove incentives for the new policy to get far from the old policy.
 
<sup>1</sup><span id="fn1"></span>Referenced from [OpenAI](https://spinningup.openai.com/en/latest/algorithms/ppo.html) 

### 5.2. PPO-Clip mathematical model

$$ \begin{equation}\mathbf{
 L^{PPO} (\theta)=\mathbb{\hat{E}}_t\:[L^{CLIP}(\theta)-c_1L^{VF}(\theta)+c_2S[\pi_\theta](s_t)]}
 \end{equation}$$ 
 
1. $ L^{CLIP} (\theta)=\mathbb{\hat{E}}_t[\min(r_t(\theta)\:\hat{A}^t,\:\:clip(r_t(\theta),\:\:1-\epsilon,\:\:1+\epsilon)\hat{A}^t)]$ 
<br>*where*,
* $r_t(\theta)\:\hat{A}^t$: Surrogate objective is the probability ratio between a new policy network and an older policy network.

* $\epsilon$: Hyper-parameter; usually with a value of 0.2.

* clip$(r_t(\theta),\:\:1-\epsilon,\:\:1+\epsilon)\:\hat{A}^t$: Clipped version of the surrogate objective, where the probability ratio is truncated.

2. $c_1L^{VF}(\theta)$: Determines desirability of the current state.

3. $c_2S[\pi_\theta](s_t)$: The entropy term using Gaussian Distribution.

In [18]:
# necessary imports
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
# import torch.nn.functional as F
#from torch.distributions import Normal

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

### 5.3. Neural Network

In [19]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)
        

class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size, std=0.0):
        super(ActorCritic, self).__init__()
        
        self.critic = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs),
        )
        self.log_std = nn.Parameter(torch.ones(1, num_outputs) * std)
        
        self.apply(init_weights)
        
    def forward(self, x):
        value = self.critic(x)
        mu    = self.actor(x)
        std   = self.log_std.exp().expand_as(mu)
        dist  = Normal(mu, std)
        return dist, value

In [20]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()
    
def test_env(vis=False):
    state = env.reset()
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        next_state, reward, done, _ = env.step(dist.sample().cpu().numpy()[0])
        state = next_state
        if vis: env.render()
        total_reward += reward
    return total_reward